In [1]:
import pandas as pd
import glob
import numpy as np
import cloudscraper
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.display import clear_output

manualColumns = ['Background', 
                'Hair Style', 
                'Hair Color', 
                'Facial Hair', 
                'Top', 
                'Bottom',
                'Footwear',
                'Ring',
                'Headwear',
                'Glasses',
                'Piercings', 
                'Accessories']

In [9]:
projectFiles = ['./Attributes_1_5000.csv',
'./Attributes_5001_10000.csv',
'./Attributes_10001_15000.csv',
'./Attributes_15001_20000.csv',
'./Attributes_20001_25000.csv',
'./Attributes_25001_30000.csv', 
'./Attributes_30001_35000.csv',
'./Attributes_35001_40000.csv',
'./Attributes_40001_45000.csv', 
'./Attributes_45001_50000.csv', 
'./Attributes_50001_55000.csv', 
'./Attributes_55001_60000.csv',
'./Attributes_60001_65000.csv',
'./Attributes_65001_70000.csv',
'./Attributes_70001_75000.csv',
'./Attributes_75001_80000.csv',
'./Attributes_80001_85000.csv',
'./Attributes_85001_90000.csv',
'./Attributes_90001_95000.csv',]
#'./Attributes_95000_99999.csv']

#df = pd.concat(map(pd.read_csv, glob.glob('./Attributes_1*.csv')))
df = pd.concat(map(pd.read_csv, projectFiles))

In [10]:
mci = manualColumns.copy()
mci.insert(0,'ID')
df.columns = mci

In [11]:
validCount = len(df.index) - len(np.where(pd.isnull(df['Background']))[0])
print(validCount)

94860


In [12]:
missingIDs = np.where(pd.isnull(df['Background']))[0]
print(missingIDs)

[28383 29784 33922 33923 34195 34449 34701 34702 34711 34775 34776 34779
 34786 34793 34794 34854 34855 34885 34886 34887 34893 34894 34895 34896
 35095 35096 35112 35113 35120 35121 35122 35123 35124 35125 35171 35172
 35193 35202 35203 35204 35205 35211 35267 35268 35281 35282 35290 35291
 35301 35302 35303 35504 35505 35527 35529 35561 35562 35563 35564 35648
 35843 35844 35992 36000 36001 36067 36068 36069 36095 36480 36481 36486
 36487 36630 36873 36950 36965 37200 37201 37472 37473 37476 37878 38031
 38355 38357 38362 38430 38433 38632 38761 38762 38787 38851 38852 41477
 42091 42092 43405 43406 43407 43408 43460 43637 43638 43639 44468 45171
 45172 45173 45174 45175 45201 45276 45277 45278 45279 45462 45463 45464
 45466 45473 45657 45658 45659 45660 45823 45830 46178 46659 46807 47279
 47863 49067 51691 51855 51947 52204 52205 52206]


In [13]:
listing = {}

In [14]:
%%time
startRange = 1
stopRange = 100000

manualColums = ['Background', 
                'Hair Style', 
                'Hair Color', 
                'Facial Hair', 
                'Top', 
                'Bottom',
                'Footwear',
                'Ring',
                'Headwear',
                'Glasses',
                'Piercings', 
                'Accessories']

refStartx = startRange

#for x in range(startRange, stopRange):
for x in (missingIDs):
    scraper = cloudscraper.create_scraper(
        delay=1,
        browser={
            'browser': 'firefox',
            'platform': 'windows',
            'mobile': False
        })
    html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/" + str(x + 1)).text

    soup = BeautifulSoup(html, 'lxml')

    rows = soup.find_all('div', 'label')

    rowAttributes = {}
    for row in rows:
        cells = row
        str_row = str(row)
        value = str_row.split('!-- -->')
        cleanTextAttribute = BeautifulSoup(value[0], "lxml").get_text()
        if cleanTextAttribute == 'Available on PALM':
            continue
        if (len(value) > 1):
            cleanTextValue = BeautifulSoup(value[1], "lxml").get_text().strip()
        else:
            cleanTextValue = "None"
        
        rowAttributes[cleanTextAttribute] = cleanTextValue
    
    output_list = [(rowAttributes[key]) for key in (manualColums) if key in rowAttributes]

    
    listing[str(x+1)] = output_list
    clear_output(wait=True)
    print(x+1)
print(listing)
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColums)  #pd.DataFrame(listing, orient='index')
data_dictionary.to_csv('Attributes_' + str(missingIDs[0]) + '_' + str(missingIDs[len(missingIDs)-1]) + '.csv')
refStartx = x+1
listing.clear()

52207
{'28384': [], '29785': [], '33923': [], '33924': [], '34196': [], '34450': [], '34702': [], '34703': [], '34712': [], '34776': [], '34777': [], '34780': [], '34787': [], '34794': [], '34795': [], '34855': [], '34856': [], '34886': [], '34887': [], '34888': [], '34894': [], '34895': [], '34896': [], '34897': [], '35096': [], '35097': [], '35113': [], '35114': [], '35121': [], '35122': [], '35123': [], '35124': [], '35125': [], '35126': [], '35172': [], '35173': [], '35194': [], '35203': [], '35204': [], '35205': [], '35206': [], '35212': [], '35268': [], '35269': [], '35282': [], '35283': [], '35291': [], '35292': [], '35302': [], '35303': [], '35304': [], '35505': [], '35506': [], '35528': [], '35530': [], '35562': [], '35563': [], '35564': [], '35565': [], '35649': [], '35844': [], '35845': [], '35993': [], '36001': [], '36002': [], '36068': [], '36069': [], '36070': [], '36096': [], '36481': [], '36482': [], '36487': [], '36488': [], '36631': [], '36874': [], '36951': [], '3696

ValueError: 12 columns passed, passed data had 0 columns

In [16]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index')  #pd.DataFrame(listing, orient='index')

In [17]:
data_dictionary

""
28384
29785
33923
33924
34196
...
51856
51948
52205
52206


In [ ]:
for count in range(0, len(manualColums)):
    print(count)
    print(data_dictionary[manualColums[count]].value_counts())
    data_dictionary[manualColums[count]].value_counts(normalize=True)
    print('---------------------------------')

In [ ]:
data_dictionary.to_csv('Attributes_' + str(startRange) + '_' + str(stopRange-1) + '.csv')

In [ ]:
print('done')